In [2]:
"""
Dataset Generation for Tau Protein Misfolding Prediction

This notebook:
1. Loads tau protein sequences from FASTA file
2. Creates or loads labels
3. Validates data quality
4. Saves processed datasets
"""

import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Import ONLY dataset-related utilities
from utils.dataset import (
    load_fasta,
    save_fasta,
)

from utils.dataset import (
    create_synthetic_labels,
    load_labels,
    validate_sequences,
    save_core_tables,
)

from utils.config import (
    FASTA_FILE,
    SEQUENCES_CSV,
    LABELS_CSV,
    PROCESSED_DATA_DIR,
)

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ All imports successful!")



✅ All imports successful!


In [3]:
"""
Load tau protein sequences from FASTA file
"""

print("=" * 80)
print("STEP 1: LOADING FASTA FILE")
print("=" * 80)

# Check if FASTA file exists
if not FASTA_FILE.exists():
    print(f"⚠️  FASTA file not found: {FASTA_FILE}")
    print("Creating example FASTA file for demonstration...")
    
    # Create example sequences (replace with real data)
    example_sequences = pd.DataFrame({
        'protein_id': [f'TAU_{i:04d}' for i in range(100)],
        'description': ['Tau protein example'] * 100,
        'sequence': ['MAEPRQEFEVMEDHAGTYGLGDRKDQGGYTMHQDQEGDTDAGLKESPLQTP'] * 100,
        'length': [52] * 100,
        'species': ['Homo sapiens'] * 50 + ['Mus musculus'] * 50,
    })
    
    # Save example FASTA
    FASTA_FILE.parent.mkdir(parents=True, exist_ok=True)
    save_fasta(example_sequences, FASTA_FILE)
    print(f"✅ Created example FASTA: {FASTA_FILE}")

# Load FASTA
df_sequences = load_fasta(FASTA_FILE)

print(f"\n📊 Loaded {len(df_sequences)} sequences")
print(f"Columns: {df_sequences.columns.tolist()}")


2025-12-18 06:19:28,146 - utils.dataset - INFO - Loading FASTA file: /workspaces/Alzheimer-s-Biomarker/tau_stacking_project/data/raw/tau_all_species.fasta
2025-12-18 06:19:28,148 - utils.dataset - ERROR - Error loading FASTA file: No sequences found in /workspaces/Alzheimer-s-Biomarker/tau_stacking_project/data/raw/tau_all_species.fasta


STEP 1: LOADING FASTA FILE


ValueError: No sequences found in /workspaces/Alzheimer-s-Biomarker/tau_stacking_project/data/raw/tau_all_species.fasta

In [ ]:
"""
Explore the loaded sequences
"""

print("=" * 80)
print("STEP 2: DATA EXPLORATION")
print("=" * 80)

# Display first few sequences
print("\n📋 First 5 sequences:")
print(df_sequences.head())

# Basic statistics
print("\n📊 Sequence length statistics:")
print(df_sequences['length'].describe())

# Species distribution
print("\n🌍 Species distribution:")
print(df_sequences['species'].value_counts())

# Visualize sequence length distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(df_sequences['length'], bins=30, color='skyblue', edgecolor='black')
axes[0].set_xlabel('Sequence Length')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Sequence Lengths')
axes[0].grid(alpha=0.3)

# Box plot
axes[1].boxplot(df_sequences['length'])
axes[1].set_ylabel('Sequence Length')
axes[1].set_title('Sequence Length Box Plot')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n✅ Sequence length range: {df_sequences['length'].min()} - {df_sequences['length'].max()}")


In [ ]:
"""
Validate protein sequences for quality
"""

print("=" * 80)
print("STEP 3: SEQUENCE VALIDATION")
print("=" * 80)

# Validate sequences
is_valid, errors = validate_sequences(df_sequences, sequence_column='sequence')

if is_valid:
    print("✅ All sequences are valid!")
else:
    print(f"⚠️  Found {len(errors)} validation errors:")
    for i, error in enumerate(errors[:10]):  # Show first 10 errors
        print(f"  {i+1}. {error}")
    
    if len(errors) > 10:
        print(f"  ... and {len(errors) - 10} more errors")

# Check amino acid composition
print("\n🔬 Checking amino acid composition...")
all_amino_acids = ''.join(df_sequences['sequence'].tolist())
unique_chars = set(all_amino_acids)
valid_aa = set('ACDEFGHIKLMNPQRSTVWY')

print(f"Unique characters found: {sorted(unique_chars)}")
invalid_chars = unique_chars - valid_aa
if invalid_chars:
    print(f"⚠️  Invalid characters: {invalid_chars}")
else:
    print("✅ All characters are valid amino acids")

# Calculate amino acid frequency
aa_counts = {aa: all_amino_acids.count(aa) for aa in valid_aa}
total_aa = sum(aa_counts.values())
aa_freq = {aa: count/total_aa*100 for aa, count in aa_counts.items()}

# Plot amino acid frequency
plt.figure(figsize=(14, 5))
sorted_aa = sorted(aa_freq.items(), key=lambda x: x[1], reverse=True)
aas, freqs = zip(*sorted_aa)

plt.bar(aas, freqs, color='steelblue', edgecolor='black')
plt.xlabel('Amino Acid')
plt.ylabel('Frequency (%)')
plt.title('Amino Acid Composition Across All Sequences')
plt.grid(alpha=0.3, axis='y')
plt.tight_layout()
plt.show()


In [ ]:
"""
Create or load labels for sequences
"""

print("=" * 80)
print("STEP 4: LABELS")
print("=" * 80)

# Check if labels file exists
if LABELS_CSV.exists():
    print(f"Loading existing labels from: {LABELS_CSV}")
    df_labels = load_labels(LABELS_CSV)
else:
    print("⚠️  Labels file not found. Creating synthetic labels for demonstration...")
    print("❗ IMPORTANT: Replace with real labels for actual research!")
    
    # Create synthetic labels
    df_labels = create_synthetic_labels(df_sequences, positive_ratio=0.3)
    
    # Save synthetic labels
    LABELS_CSV.parent.mkdir(parents=True, exist_ok=True)
    df_labels.to_csv(LABELS_CSV, index=False)
    print(f"✅ Saved synthetic labels to: {LABELS_CSV}")

# Display label statistics
print(f"\n📊 Label statistics:")
print(df_labels.head())
print(f"\nTotal labels: {len(df_labels)}")
print(f"\nLabel distribution:")
print(df_labels['label'].value_counts())
print(f"\nLabel proportions:")
print(df_labels['label'].value_counts(normalize=True))

# Visualize label distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Count plot
label_counts = df_labels['label'].value_counts()
axes[0].bar(['Normal (0)', 'Misfolding (1)'], 
            [label_counts[0], label_counts[1]], 
            color=['lightgreen', 'salmon'],
            edgecolor='black')
axes[0].set_ylabel('Count')
axes[0].set_title('Label Distribution (Counts)')
axes[0].grid(alpha=0.3, axis='y')

# Pie chart
axes[1].pie([label_counts[0], label_counts[1]], 
            labels=['Normal (0)', 'Misfolding (1)'],
            colors=['lightgreen', 'salmon'],
            autopct='%1.1f%%',
            startangle=90)
axes[1].set_title('Label Distribution (Percentage)')

plt.tight_layout()
plt.show()


In [ ]:
"""
Merge sequences with labels
"""

print("=" * 80)
print("STEP 5: MERGE SEQUENCES AND LABELS")
print("=" * 80)

# Merge dataframes
df_merged = df_sequences.merge(df_labels, on='protein_id', how='inner')

print(f"📊 Merged dataset:")
print(f"  Total samples: {len(df_merged)}")
print(f"  Features: {df_merged.columns.tolist()}")

# Check for missing values
print(f"\n🔍 Missing values:")
print(df_merged.isnull().sum())

# Display merged data
print(f"\n📋 First 5 rows of merged data:")
print(df_merged.head())

# Analyze sequence length by label
print(f"\n📏 Sequence length by label:")
print(df_merged.groupby('label')['length'].describe())

# Visualize
fig, ax = plt.subplots(figsize=(10, 5))
df_merged.boxplot(column='length', by='label', ax=ax)
ax.set_xlabel('Label (0=Normal, 1=Misfolding)')
ax.set_ylabel('Sequence Length')
ax.set_title('Sequence Length Distribution by Label')
plt.suptitle('')  # Remove default title
plt.tight_layout()
plt.show()


In [ ]:
"""
Save processed sequences and labels
"""

print("=" * 80)
print("STEP 6: SAVE PROCESSED DATA")
print("=" * 80)

# Prepare data for saving
df_sequences_final = df_merged[['protein_id', 'description', 'sequence', 'length', 'species']]
df_labels_final = df_merged[['protein_id', 'label', 'source']]

# Create dummy splits table (will be properly split in next notebook)
df_splits_temp = pd.DataFrame({
    'protein_id': df_merged['protein_id'],
    'split': 'none'  # Placeholder
})

# Save
PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)

df_sequences_final.to_csv(SEQUENCES_CSV, index=False)
df_labels_final.to_csv(LABELS_CSV, index=False)

print(f"✅ Saved sequences to: {SEQUENCES_CSV}")
print(f"✅ Saved labels to: {LABELS_CSV}")

print(f"\n📊 Summary:")
print(f"  Total sequences: {len(df_sequences_final)}")
print(f"  Total labels: {len(df_labels_final)}")
print(f"  Sequence length: {df_sequences_final['length'].min()} - {df_sequences_final['length'].max()}")
print(f"  Label 0 (Normal): {(df_labels_final['label']==0).sum()}")
print(f"  Label 1 (Misfolding): {(df_labels_final['label']==1).sum()}")


In [ ]:
"""
Summary and next steps
"""

print("=" * 80)
print("✅ DATASET GENERATION COMPLETE!")
print("=" * 80)

print("\n📋 Generated Files:")
print(f"  1. {SEQUENCES_CSV}")
print(f"  2. {LABELS_CSV}")

print("\n📊 Dataset Statistics:")
print(f"  Total samples: {len(df_merged)}")
print(f"  Features: {len(df_merged.columns)}")
print(f"  Avg sequence length: {df_merged['length'].mean():.1f}")
print(f"  Class balance: {(df_merged['label']==1).mean()*100:.1f}% positive")

print("\n🎯 Next Steps:")
print("  → Run notebook 02_preprocessing.ipynb to:")
print("     - Create train/val/test splits")
print("     - Generate embeddings")
print("     - Encode sequences")

print("\n" + "=" * 80)
